In [2]:
from selenium import webdriver

import requests
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup, element
import time
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from selenium.webdriver.common.action_chains import ActionChains

In [51]:
# Define the search query
location = input("Enter the location: ")
min_price = input("Enter the minimum price: ")
max_price = input("Enter the maximum price: ")
min_area = input("Enter the minimum area: ")
max_area = input("Enter the maximum area: ")
distance = input("Enter the maximum distance from location in km: ")
listing_type = input("Enter listing type (mieszkanie/dom/dzialka): ")
offer_type = input("Enter offer type (sprzedaz/wynajem): ")

# Start the webdriver and load the website
url = f"https://www.otodom.pl/{offer_type}/{listing_type}/{location}/?search%5Bfilter_float_price%3Afrom%5D={min_price}&search%5Bfilter_float_price%3Ato%5D={max_price}&search%5Bfilter_float_m%3Afrom%5D={min_area}&search%5Bfilter_float_m%3Ato%5D={max_area}&search%5Bdist%5D={distance}"
driver = webdriver.Chrome()
driver.get(url)

# Close Cookies banner
cookie_banner = driver.find_element(By.ID,'onetrust-accept-btn-handler')
cookie_banner.click()



In [57]:
# Define the search query
search_params = {
    "Location": input("Enter the location: "),
    "Min Price": input("Enter the minimum price: "),
    "Max Price": input("Enter the maximum price: "),
    "Min Area": input("Enter the minimum area: "),
    "Max Area": input("Enter the maximum area: "),
    "Max Distance (km)": input("Enter the maximum distance from location in km: "),
    "Listing Type": input("Enter listing type (mieszkanie/dom/dzialka): "),
    "Offer Type": input("Enter offer type (sprzedaz/wynajem): ")
}

# Start the webdriver and load the website
url = f"https://www.otodom.pl/{search_params['Offer Type']}/{search_params['Listing Type']}/{search_params['Location']}/?search%5Bfilter_float_price%3Afrom%5D={search_params['Min Price']}&search%5Bfilter_float_price%3Ato%5D={search_params['Max Price']}&search%5Bfilter_float_m%3Afrom%5D={search_params['Min Area']}&search%5Bfilter_float_m%3Ato%5D={search_params['Max Area']}&search%5Bdist%5D={search_params['Max Distance (km)']}"
driver = webdriver.Chrome()
driver.get(url)

# Close Cookies banner
cookie_banner = driver.find_element(By.ID,'onetrust-accept-btn-handler')
cookie_banner.click()


In [58]:
# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(driver.page_source)


# Find all the relevant information
records = soup.find_all("li", {'class': 'css-iq9jxc e1n6ljqa1'})

In [59]:
records

[<li class="css-iq9jxc e1n6ljqa1" data-cy="listing-item"><a class="css-1up0y1q e1n6ljqa16" data-cy="listing-item-link" href="/pl/oferta/mieszkanie-2-pokojowe-gdynia-maly-kack-ID4kyYJ"><aside class="css-qou0ww e1n6ljqa3"><picture><source data-cy="listing-item-image-source" media="(max-width: 768px)" srcset="https://ireland.apollo.olxcdn.com/v1/files/eyJmbiI6IjQ5bnE5azhjYXA5bjMtQVBMIiwidyI6W3siZm4iOiJlbnZmcXFlMWF5NGsxLUFQTCIsInMiOiIxNCIsInAiOiIxMCwtMTAiLCJhIjoiMCJ9XX0.J4n7gSdN_DBPmJbD5NoGwrmWVh9NsmStmAdcXNFlZFA/image;s=655x491;q=80"><source data-cy="listing-item-image-source" media="(min-width: 768px)" srcset="https://ireland.apollo.olxcdn.com/v1/files/eyJmbiI6IjQ5bnE5azhjYXA5bjMtQVBMIiwidyI6W3siZm4iOiJlbnZmcXFlMWF5NGsxLUFQTCIsInMiOiIxNCIsInAiOiIxMCwtMTAiLCJhIjoiMCJ9XX0.J4n7gSdN_DBPmJbD5NoGwrmWVh9NsmStmAdcXNFlZFA/image;s=1280x1024;q=80"><img alt="Mieszkanie 2 pokojowe, Gdynia - Mały Kack" class="css-s6l5j9 e1xjv2fs0" height="210" loading="lazy" src="https://ireland.apollo.olxcdn.com/v1/f

In [60]:
#this gets the Top 13 results only
data = []

for record in records:
    # Extract the link to the record
    link = record.find("a", {'class':'css-1up0y1q e1n6ljqa16'}).get("href")
    link = "https://www.otodom.pl/"+link

    # Extract the title
    property_title = record.find("h3", {'class' :'css-1mmyqk7 e1n6ljqa6'}).get("title")

    # Extract the address
    address =record.find("p", {'class' :'css-14aokuk e1n6ljqa7'}).get("title")    

    # Extract the number of rooms, area, and price
    details = record.find_all("span", {'class':'css-1ntk0hg ei6hyam1'})
    rooms = ""
    area = ""
    price = ""
    for detail in details:
        if "pokoje" in detail.text:
            rooms = detail.text.strip()#.replace(" pokoje", "")
        elif "m²" in detail.text:
            area = detail.text.strip()#.replace(" m²", "")
        elif "zł/mc" in detail.text:
            price = detail.text.strip()#.replace("zł/mc", "")

    # Append the attributes to the data list
    data.append({
        "Link": link,
        "Property": property_title,
        "Address ": address,
        "Rooms": rooms,
        "Area (m²)": area,
        "Price (zł/mc)": price
    })



# Create a dataframe from the data list
df = pd.DataFrame(data)
#df["Rooms"] = df["Rooms"].astype(float)
#df["Area (m²)"] = df["Area (m²)"].astype(float)
#df["Price (zł/mc)"] = df["Price (zł/mc)"].astype(float)

In [61]:
df

,Link,Property,Address,Rooms,Area (m²),Price (zł/mc)
0,https://www.otodom.pl//pl/oferta/mieszkanie-2-...,"Mieszkanie 2 pokojowe, Gdynia - Mały Kack","Mieszkanie na wynajem: Gdynia, Mały Kack, ul. ...",2 pokoje,44 m²,2990 zł/mc
1,https://www.otodom.pl//pl/oferta/dwupokojowe-z...,Dwupokojowe z widokiem na morze- Dolny Sopot,"Mieszkanie na wynajem: Sopot, Dolny, Pokorniew...",2 pokoje,41 m²,2700 zł/mc
2,https://www.otodom.pl//pl/oferta/sloneczne-mie...,Słoneczne mieszkanie z urzekającym widokiem,"Mieszkanie na wynajem: Gdańsk, Suchanino, Nosk...",2 pokoje,51.5 m²,2790 zł/mc
3,https://www.otodom.pl//pl/oferta/mieszaknie-gd...,Mieszaknie Gdańsk Świętkorzyska 2 pokoje 41m2,"Mieszkanie na wynajem: Gdańsk, pomorskie",2 pokoje,41 m²,2400 zł/mc
4,https://www.otodom.pl//pl/oferta/mieszkanie-na...,Mieszkanie na wynajem (2 pokoje) - przymorze,"Mieszkanie na wynajem: Gdańsk, Przymorze, ul. ...",2 pokoje,50 m²,2700 zł/mc
5,https://www.otodom.pl//pl/oferta/nowe-rewelacy...,nowe rewelacyjne mieszkanie Gdańsk Osowa 2 pokoje,"Mieszkanie na wynajem: Gdańsk, Osowa, ul. Kozi...",2 pokoje,43 m²,2350 zł/mc
6,https://www.otodom.pl//pl/oferta/mieszkanie-dw...,Mieszkanie dwa oddzielne pokoje z garażem,"Mieszkanie na wynajem: Gdańsk, Jasień, ul. Law...",2 pokoje,45 m²,2650 zł/mc
7,https://www.otodom.pl//pl/oferta/nowe-2-pokojo...,Nowe 2-pokojowe mieszkanie do wynajęcia od zaraz!,"Mieszkanie na wynajem: Gdańsk, Oliwa, Home by ...",2 pokoje,42.5 m²,2850 zł/mc
8,https://www.otodom.pl//pl/oferta/mieszkanie-2-...,Mieszkanie 2 pokoje NOWE! Zwierzęta mile widzi...,"Mieszkanie na wynajem: Gdańsk, Oliwa, ul. Kazi...",2 pokoje,43.4 m²,3350 zł/mc
9,https://www.otodom.pl//pl/oferta/wynajme-miesz...,Wynajmę mieszkanie 2 pokojowe,"Mieszkanie na wynajem: Rotmanka, gdański, pomo...",2 pokoje,42 m²,2000 zł/mc
